In [59]:

import pandas as pd
import pandas_datareader.data as web
from pandas import Series, DataFrame
import datetime
from datetime import date, timedelta, datetime
import math
import matplotlib as mpl
import numpy as np
import talib as ta
import yfinance as yf
from tqdm import tqdm
from os import walk


In [61]:
def buyStock(row):
    close_adj = row['Adj Close']
    starting_price = close_adj
    return close_adj,starting_price,True
    

In [62]:
def sellStock(row,starting_price):
    close_adj = row['Adj Close']
    single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
    return close_adj,single_trade_percent_gain,False
    

In [66]:
def trade(stock,startDate,endDate): 
    StartingAmount = 500_000/505

    df = pd.read_csv("C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/filter_30y_stock_csvs/{}.csv".format(stock))
    df = df.rename(columns={"CLOSE": "Adj Close", "VOLUME": "Volume"})

    df['Datetime'] = pd.to_datetime(df['Datetime'])

    df = df[~(df['Datetime'] < startDate)]
    df = df[~(df['Datetime'] > endDate)]

    StockData = pd.DataFrame(df).to_numpy()

    Total_Gain =0
    starting_price =1

    Start_Price = (df['Adj Close'].head(1))
    Start_Price = float(Start_Price)

    End_Price = (df['Adj Close'].tail(1))
    End_Price = float(End_Price)

    TotalReturn = StartingAmount
    index =0
    bought = False




    # CONVERT EVERYTHING INTO NUMPY
    for date_var,row in df.iterrows():
        maxValue = df.iloc[index-5:index]['Adj Close'].max()
        if math.isnan(maxValue):
            maxValue = 0
            
        if not bought:
            if(df.iloc[index-1]['ADX']>15): #If trend is appearing - See if its a good time to buy
                if df.iloc[index-1]['Volume Higher than MAVG']==1 and df.iloc[index-1]['Price Lower than MAVG']==1: 
                        close_adj,starting_price,bought = buyStock(row)

                # elif df.iloc[index-1]['Adj Close'] > maxValue and maxValue != 0: #This doesnt work -- edit and relook
                #     close_adj,starting_price,bought = buyStock(row)
            
                # elif df.iloc[index-1]['RSI'] > 70:
                #     close_adj,starting_price,bought = buyStock(row)
               
        elif bought and close_adj != row['Adj Close']: #This is sell clause
            if (df.iloc[index-1]['Volume Higher than MAVG']==0 or df.iloc[index-1]['RSI'] < 35) and df.iloc[index-1]['Price Lower than MAVG']==0 :
                   
                    infoOnSelling = sellStock(row,starting_price)
                    close_adj = infoOnSelling[0]
                    Total_Gain += infoOnSelling[1]
                    bought = infoOnSelling[2]

                    
            # elif (((df.iloc[index-1]['Adj Close']- starting_price)/starting_price) * 100) > (.5 * 100):
                   
            #         infoOnSelling = sellStock(row,starting_price)
            #         close_adj = infoOnSelling[0]
            #         Total_Gain += infoOnSelling[1]
            #         bought = infoOnSelling[2]

                    

            # elif df.iloc[index-1]['Adj Close'] < maxValue and maxValue != 0: 
                    
            #         infoOnSelling = sellStock(row,starting_price)
            #         close_adj = infoOnSelling[0]
            #         Total_Gain += infoOnSelling[1]
            #         bought = infoOnSelling[2]

        index +=1
        TotalReturn = (1 + Total_Gain) * TotalReturn
        Total_Gain = 0


    
    if bought:
        infoOnSelling = sellStock(row,starting_price)
        close_adj = infoOnSelling[0]
        Total_Gain += infoOnSelling[1]
        bought = infoOnSelling[2]
        
        TotalReturn = (1+Total_Gain)*TotalReturn
        Total_Gain = 0

    Total_Gain = ((TotalReturn - StartingAmount)/StartingAmount) 
    return round(TotalReturn-StartingAmount,2)


In [69]:
f = []
ROIall = {}
money = {}
startDate = datetime.strptime('2021-09-01',"%Y-%m-%d")
offsetDate = timedelta(39)
endDate = datetime.strptime('2021-11-01',"%Y-%m-%d")
sum = 0
printStyle = "."
startAmt = 500_000/505
for (dirpath, dirnames, filenames) in walk("C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/clean_30y_stock_csvs/"):
    f.extend(filenames)
    break


for stock in tqdm(f):
    stock = stock[:-4]
    money[stock] = trade(stock,startDate - offsetDate,endDate)
    #print("{} is done with profit of: {} which is a percent of: {}%".format(stock,money[stock],round((money[stock]/startAmt)*100,2)))
    #print(".", end=" ")

for s in money:
    sum+= money[s]

percentageOfWin = ((sum)/500_000) * 100
print()
print("Total win of ${} with % of {}%".format(round(sum,2),round(percentageOfWin,2)))


100%|██████████| 505/505 [00:35<00:00, 14.10it/s]


Total win of $9390.28 with % of 1.88%


In [70]:
sp500 = yf.download('^GSPC',startDate,endDate)
profitsp500percentage = ((sp500['Close'][-1] - sp500['Close'][0])/sp500['Close'][0])*100
profitsp500 = (500_000) * (profitsp500percentage/100)

print("This is SAP 500 : {}% and the increase in money: {}".format(round(profitsp500percentage,2),round(profitsp500,2)))

[*********************100%***********************]  1 of 1 completed
This is SAP 500 : 1.8% and the increase in money: 8984.13
